In [1]:
import csv
import json
import numpy as np
import pandas as pd
import country_converter as coco

In [4]:
fp = "/Users/romain/GitHub/edges/dev/GeoPolRisk/geopolrisk_paired_cfs_v2.xlsx"

In [5]:
# Load the data into a DataFrame
df = pd.read_excel(fp, sheet_name="Sheet1")

In [6]:
country_map = {
    c: coco.convert(c, to="iso2") for c in df["Importing Country"].unique()
}

country_map.update(
    {c: coco.convert(c, to="iso2") for c in df["Exporting Country"].unique()}
)

for k, v in country_map.items():
    if v == "not found":
        print(k)

country_map = {k: v for k, v in country_map.items() if v != "not found"}

Other Asia, nes not found in regex
Global not found in regex
Other Asia, nes not found in regex


Other Asia, nes
Global


In [7]:
data = []

df["Exporting Location"] = df["Exporting Country"].map(country_map).fillna("GLO")
df["Importing Location"] = df["Importing Country"].map(country_map).fillna("GLO")


grouped = df.groupby(["Resource Name", "Exporting Country", "Importing Country"])

for (resource, export_country, importing_country), group in grouped:
    if not group["GeoPolRisk Characterization Factor Normalized to copper [-]"].empty:
        data.append({
            "supplier": {
                "name": group["Dataset name"].iloc[0],
                "reference product": group["Dataset reference product"].iloc[0],
                "location": group["Exporting Location"].iloc[0],
                "operator": group["operator"].iloc[0],
                "matrix": "technosphere"
            },
            "consumer": {
                "location": group["Importing Location"].iloc[0],
                "matrix": "technosphere"
            },
            "value": group["GeoPolRisk Characterization Factor Normalized to copper [-]"].mean(),
        })


In [8]:
len(data)

53030

In [9]:
data[0]

{'supplier': {'name': 'aluminium production, primary',
  'reference product': 'aluminium, primary',
  'location': 'AU',
  'operator': 'startswith',
  'matrix': 'technosphere'},
 'consumer': {'location': 'CA', 'matrix': 'technosphere'},
 'value': 1.101763446251004e-10}

In [10]:
fp_out = "/Users/romain/GitHub/edges/edges/data/GeoPolRisk_paired_2024.json"
with open(fp_out, "w") as f:
    json.dump(data, f, indent=2)